In [25]:
import cv2
import os

# 영상 불러오기
video_path = os.path.expanduser('~/Documents/ds_study/mlb hitter/strider2.mp4')
cap = cv2.VideoCapture(video_path)

# 홈플레이트의 좌우 좌표를 저장할 리스트
home_plate_width_coords = []

# 클릭을 이용해 처리할 함수
def click_event(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        home_plate_width_coords.append((x, y))
        cv2.circle(frame, (x, y), 5, (0, 255, 0), -1)
        if len(home_plate_width_coords) == 2:  # 두 점이 선택되면 선을 그림
            cv2.line(frame, home_plate_width_coords[0], home_plate_width_coords[1], (0, 255, 0), 2)
        cv2.imshow("Frame", frame)

# 영상의 첫 프레임  가져오기
ret, frame = cap.read()
if not ret:
    print("Failed to grab the first frame of the video.")
    cap.release()
    cv2.destroyAllWindows()
    exit()

# 클릭 할 수 있는 창을 생성
cv2.imshow("Frame", frame)
cv2.setMouseCallback("Frame", click_event)

cv2.waitKey(0)
cap.release()
cv2.destroyAllWindows()

# 클릭한 좌우 좌표 출력
print("Selected Width Coordinates: ", home_plate_width_coords)

Selected Width Coordinates:  [(611, 334), (675, 335)]


In [13]:
import cv2
import mediapipe as mp
import numpy as np
import os

# MediaPipe 초기화
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Pose 모델을 초기화
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)

# 스트라이크 존을 시각화하는 함수
def draw_strike_zone(frame, strike_zone_top, strike_zone_bottom, strike_zone_left, strike_zone_right):
    cv2.rectangle(frame, (strike_zone_left, strike_zone_top), (strike_zone_right, strike_zone_bottom), (0, 255, 0), 2)

# 초기 ROI 설정
roi = None

# 마우스 콜백 함수
def select_roi(event, x, y, flags, param):
    global roi, start_point
    if event == cv2.EVENT_LBUTTONDOWN:
        roi = None  # ROI 초기화
        start_point = (x, y)  # 시작점 저장
    elif event == cv2.EVENT_LBUTTONUP:
        # 마우스 드래그가 끝나는 지점에서 ROI 설정
        roi = (min(start_point[0], x), min(start_point[1], y), abs(start_point[0] - x), abs(start_point[1] - y))

# 스트라이크 존을 계산하는 함수입니다.
def calculate_strike_zone(shoulder, hip, knee, home_plate_width_coords):
    mid_shoulder_hip = round((shoulder[1] + hip[1]) / 2)  # 평균을 구한 뒤 반올림
    strike_zone_top = mid_shoulder_hip
    strike_zone_bottom = round(knee[1])  # 반올림
    strike_zone_left = home_plate_width_coords[0][0]  # 왼쪽 경계 (반올림 없음)
    strike_zone_right = home_plate_width_coords[1][0]  # 오른쪽 경계 (반올림 없음)
    return (strike_zone_top, strike_zone_bottom, strike_zone_left, strike_zone_right)

# 홈플레이트의 좌우경계 좌표를 정의합니다.
home_plate_coords = [(610, 334), (673, 334)]  

# 영상 초기화
video_path = os.path.expanduser('~/Documents/ds_study/mlb hitter/strider2.mp4')
cap = cv2.VideoCapture(video_path)

cv2.namedWindow('Pose within ROI')
cv2.setMouseCallback('Pose within ROI', select_roi)

# 스트라이크 존과 타자 포즈 표시 여부를 결정하는 변수들초기화
show_strike_zone = False
current_strike_zone = None

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    if roi:
        roi_frame = frame[roi[1]:roi[1]+roi[3], roi[0]:roi[0]+roi[2]]
        results = pose.process(cv2.cvtColor(roi_frame, cv2.COLOR_BGR2RGB))

        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            # 스트라이크 존을 계산하기 위한 랜드마크 추출
            shoulder = [
                np.mean([landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x]) * frame.shape[1],
                np.mean([landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]) * frame.shape[0]
            ]
            hip = [
                np.mean([landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x]) * frame.shape[1],
                np.mean([landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]) * frame.shape[0]
            ]
            knee = [
                np.mean([landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x]) * frame.shape[1],
                np.mean([landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]) * frame.shape[0]
            ]
            
            # 스트라이크 존 계산
            if current_strike_zone is None:
                current_strike_zone = calculate_strike_zone(shoulder, hip, knee, home_plate_coords)

            mp_drawing.draw_landmarks(roi_frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            frame[roi[1]:roi[1]+roi[3], roi[0]:roi[0]+roi[2]] = roi_frame

        # ROI표시
        cv2.rectangle(frame, (roi[0], roi[1]), (roi[0] + roi[2], roi[1] + roi[3]), (0, 255, 0), 2)

    if show_strike_zone and current_strike_zone:
        draw_strike_zone(frame, *current_strike_zone)

    cv2.imshow('Pose within ROI', frame)

    key = cv2.waitKey(20) & 0xFF
    if key == ord('q'):
        break
    elif key == ord('w'):
        show_strike_zone = not show_strike_zone
    elif key == ord('e'):
        current_strike_zone = None  # 스트라이크 존을 리셋합니다.

cap.release()
cv2.destroyAllWindows()


In [26]:
import cv2
import mediapipe as mp
import numpy as np
import os

# MediaPipe 라이브러리 초기화
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Pose 모델을 초기화합니다.
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)

# 스트라이크 존을 시각화하는 함수
def draw_strike_zone(frame, strike_zone_top, strike_zone_bottom, strike_zone_left, strike_zone_right):
    cv2.rectangle(frame, (strike_zone_left, strike_zone_top), (strike_zone_right, strike_zone_bottom), (0, 255, 0), 2)

# 초기 ROI 설정. 사용자가 드래그하여 업데이트할 수 있습니다.
roi = None

# 마우스 콜백 함수입니다.
def select_roi(event, x, y, flags, param):
    global roi, start_point
    if event == cv2.EVENT_LBUTTONDOWN:
        roi = None  # ROI 초기화
        start_point = (x, y)  # 시작점 저장
    elif event == cv2.EVENT_LBUTTONUP:
        roi = (min(start_point[0], x), min(start_point[1], y), abs(start_point[0] - x), abs(start_point[1] - y))

def calculate_strike_zone(shoulder, hip, knee, home_plate_coords, frame_width, frame_height):
    #상단 경계는 어깨와 골반의 중간 지점
    strike_zone_top = int((shoulder[1] + hip[1]) / 2)
    #키에 대한 비율을 프레임의 높이로 환산
    player_height_in_pixels = hip[1] - shoulder[1]
    knee_to_bottom_ratio = 0.2  # 무릎 아래 스트라이크 존 하단까지의 계산한 비율
    knee_to_bottom_pixels = int(player_height_in_pixels * knee_to_bottom_ratio)
    # 하단 경계를 무릎의 y좌표 아래로 설정
    strike_zone_bottom = int(knee[1] + knee_to_bottom_pixels)
    # 홈플레이트의 좌우 경계는 홈플레이트의 좌표로 설정
    strike_zone_left = home_plate_coords[0][0]
    strike_zone_right = home_plate_coords[1][0]
    #스트라이크 존의 좌우 및 상하 경계가 화면을 벗어나지 않도록 함
    strike_zone_left = max(0, strike_zone_left)
    strike_zone_right = min(frame_width, strike_zone_right)
    strike_zone_top = max(0, strike_zone_top)
    strike_zone_bottom = min(frame_height, strike_zone_bottom)
    
    return (strike_zone_top, strike_zone_bottom, strike_zone_left, strike_zone_right)


# 홈플레이트의 좌우경계 좌표를 정의합니다.
home_plate_coords = [(610, 334), (673, 334)]  

# 비디오 캡처를 초기화합니다.
video_path = os.path.expanduser('~/Documents/ds_study/mlb hitter/strider2.mp4')
cap = cv2.VideoCapture(video_path)

cv2.namedWindow('Pose within ROI')
cv2.setMouseCallback('Pose within ROI', select_roi)

# 스트라이크 존과 타자 포즈 표시 여부를 결정하는 변수들을 초기화합니다.
show_strike_zone = False
current_strike_zone = None

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 프레임의 너비와 높이
    frame_height, frame_width = frame.shape[:2]

    if roi:
        roi_frame = frame[roi[1]:roi[1]+roi[3], roi[0]:roi[0]+roi[2]]
        results = pose.process(cv2.cvtColor(roi_frame, cv2.COLOR_BGR2RGB))

        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            # 선수의 랜드마크 추출
            shoulder = [
                np.mean([landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, 
                         landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x]) * frame_width,
                np.mean([landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y, 
                         landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]) * frame_height
            ]
            hip = [
                np.mean([landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, 
                         landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x]) * frame_width,
                np.mean([landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y, 
                         landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]) * frame_height
            ]
            knee = [
                np.mean([landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, 
                         landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x]) * frame_width,
                np.mean([landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y, 
                         landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]) * frame_height
            ]

            # 스트라이크 존을 계산
            if current_strike_zone is None:
                current_strike_zone = calculate_strike_zone(shoulder, hip, knee, home_plate_coords, frame_width, frame_height)

            mp_drawing.draw_landmarks(roi_frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            frame[roi[1]:roi[1]+roi[3], roi[0]:roi[0]+roi[2]] = roi_frame

        # ROI를 표시
        cv2.rectangle(frame, (roi[0], roi[1]), (roi[0] + roi[2], roi[1] + roi[3]), (0, 255, 0), 2)

    if show_strike_zone and current_strike_zone:
        draw_strike_zone(frame, *current_strike_zone)

    cv2.imshow('Pose within ROI', frame)

    key = cv2.waitKey(20) & 0xFF
    if key == ord('q'):
        break
    elif key == ord('w'):
        show_strike_zone = not show_strike_zone
    elif key == ord('e'):
        current_strike_zone = None  # 스트라이크 존 리셋

cap.release()
cv2.destroyAllWindows()

